In [9]:
import pandas as pd
import numpy as np

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
import datetime as dt

import plotly.graph_objects as go

In [80]:
ticker = "RELIANCE.NS"
start_date = dt.datetime(2020,1,1)
end_date = dt.datetime.now()
raw_data= yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [81]:
raw_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,1503.745972,1512.760498,1491.363403,1495.424927,1480.595825,6463060
2020-01-02,1497.802368,1526.480469,1497.802368,1520.883545,1505.802124,8173308
2020-01-03,1518.605103,1527.173950,1508.699097,1522.716187,1507.616455,9684434
2020-01-06,1505.727173,1513.552979,1483.933838,1487.400879,1472.651367,11315596
2020-01-07,1504.736572,1520.091064,1499.288208,1510.284058,1495.307617,7699489


In [82]:
fig = px.line(raw_data, x=raw_data.index, y="Close")
fig.show()

In [83]:
fig = go.Figure(data=[go.Candlestick(x=raw_data.index,
                                     close=raw_data['Close'],
                                     open=raw_data['Open'],
                                     high=raw_data['High'],
                                     low=raw_data['Low'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [85]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = np.average(raw_data['Volume']),
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Average Volume [2020 till date]"},
    gauge = {'axis': {'range': [None, max(raw_data['Volume'])], 'tickwidth': 1, 'tickcolor': "darkblue"},
    'bar': {'color': "darkblue"},
    'bgcolor': "#067FD0",
    }
    ))
#223BC9
fig.show()